In [37]:
import pandas as pd
from urllib.parse import urlparse
import re
import ipaddress

# Load the dataset
df = pd.read_csv('urldata.csv')

# Inspect data
df.sample(100)


,Unnamed: 0,url,label,result
71034,71034,https://www.medicalmanagementalternatives.com/,benign,0
188893,188893,https://www.facebook.com/pages/Paul-Falvo-City...,benign,0
238440,238440,https://www.lousycasino.com/,benign,0
208517,208517,https://www.hardballtimes.com/main/article/the...,benign,0
186679,186679,https://www.facebook.com/marie.c.tremblay,benign,0
...,...,...,...,...
240583,240583,https://www.mahalo.com/montreal-canadiens/,benign,0
363736,363736,http://awareness.7534ggf.siliconvalley.clearpo...,malicious,1
200435,200435,https://www.francofolies.com/artists/artist.as...,benign,0
411809,411809,http://harmathatmo.com/zapoy/gate.php/,malicious,1


In [38]:
# feature extraction preformed by URLFeatureExtraction.py
# python URLFeatureExtraction.py urldata.csv returns featuredata.csv
df = pd.read_csv('featuredata.csv', on_bad_lines='skip')

features = ['Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                        'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record',
                        'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards']

In [39]:
# Convert the features into separate columns
# df_features = pd.DataFrame(df['features'].tolist(), index=df.index, columns=features)

# Combine the extracted features with the original DataFrame
# df_combined = pd.concat([df, df_features], axis=1)
df = df.dropna(subset=['Label'])


In [40]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


# Define X (features) and y (target)
X = df[features]
y = df['Label']

# Step 1: Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Use SMOTE to balance the training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Step 3: Train the model on the resampled data
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_resampled, y_train_resampled)

# Step 4: Make predictions on the test set
y_pred = rf_model.predict(X_test)



In [ ]:
# Feature extracted URLs tested
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Step 4: Train the model
history = model.fit(X_train_resampled, y_train_resampled, epochs=5, batch_size=128, validation_data=(X_test, y_test))

# Step 5: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')



Epoch 1/5


c:\Users\Matthew\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


308/308 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6578 - loss: 0.6201 - val_accuracy: 0.7404 - val_loss: 0.5175
Epoch 2/5
308/308 ━━━━━━━━━━━━━━━━━━━━ 0s 905us/step - accuracy: 0.7280 - loss: 0.5423 - val_accuracy: 0.7404 - val_loss: 0.5059
Epoch 3/5
308/308 ━━━━━━━━━━━━━━━━━━━━ 0s 898us/step - accuracy: 0.7393 - loss: 0.5254 - val_accuracy: 0.7421 - val_loss: 0.5028
Epoch 4/5
308/308 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - accuracy: 0.7445 - loss: 0.5202 - val_accuracy: 0.7558 - val_loss: 0.4981
Epoch 5/5
308/308 ━━━━━━━━━━━━━━━━━━━━ 0s 946us/step - accuracy: 0.7455 - loss: 0.5197 - val_accuracy: 0.7441 - val_loss: 0.4976
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step - accuracy: 0.7425 - loss: 0.4977
Test Accuracy: 0.7440549731254578


In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Step 1: Prepare the data (0 for benign, 1 for phishing)

# Tokenize the URLs (character-level tokenization)
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(df['URL'])

# Convert the URLs into sequences of integers
url_sequences = tokenizer.texts_to_sequences(df['URL'])

# Pad the sequences to make them of equal length
max_sequence_length = 100  # can be adjusted based on data
X = pad_sequences(url_sequences, maxlen=max_sequence_length)

# Target labels
y = df['Label'].values

# Step 2: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Build the LSTM model
vocab_size = len(tokenizer.word_index) + 1  # Total number of unique characters
embedding_dim = 50  # Size of the character embedding vectors

model = Sequential()
# Embedding layer (convert each character to a dense vector)
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
# LSTM layer
model.add(LSTM(128, return_sequences=False))
# Output layer (binary classification)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Step 4: Train the model
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

# Step 5: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

Epoch 1/10


c:\Users\Matthew\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


278/278 ━━━━━━━━━━━━━━━━━━━━ 26s 86ms/step - accuracy: 0.8230 - loss: 0.3604 - val_accuracy: 0.9425 - val_loss: 0.1905
Epoch 2/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 23s 83ms/step - accuracy: 0.9543 - loss: 0.1600 - val_accuracy: 0.9751 - val_loss: 0.0910
Epoch 3/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 23s 83ms/step - accuracy: 0.9777 - loss: 0.0766 - val_accuracy: 0.9689 - val_loss: 0.0741
Epoch 4/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 23s 84ms/step - accuracy: 0.9821 - loss: 0.0621 - val_accuracy: 0.9930 - val_loss: 0.0251
Epoch 5/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 23s 84ms/step - accuracy: 0.9916 - loss: 0.0329 - val_accuracy: 0.9930 - val_loss: 0.0275
Epoch 6/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 24s 85ms/step - accuracy: 0.9895 - loss: 0.0395 - val_accuracy: 0.9928 - val_loss: 0.0318
Epoch 7/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 22s 80ms/step - accuracy: 0.9915 - loss: 0.0304 - val_accuracy: 0.9939 - val_loss: 0.0244
Epoch 8/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 23s 82ms/step - accuracy: 0.9812 - loss: 0.0575 - val_accurac